In [ ]:
from dev_spark_session import DevSparkSession 
from get_stock_data import GetStockData

stockdata = GetStockData() 
spark = DevSparkSession().spark


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 54293)
Traceback (most recent call last):
  File "/Users/PC/.pyenv/versions/3.11.11/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/PC/.pyenv/versions/3.11.11/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/Users/PC/.pyenv/versions/3.11.11/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/PC/.pyenv/versions/3.11.11/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/.venv/lib/python3.11/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/.venv/lib/python3.11

In [ ]:
from pyspark.sql.functions import col,explode,sha2,concat_ws,trim, lit

In [ ]:
json_data  = stockdata.get_historical_stock_data()


In [ ]:

df = spark.createDataFrame(json_data) 

df = df.withColumn("symbol", col("meta").symbol).filter( col("status") == "ok" ).drop(df.status)

df.show(truncate = False)




In [ ]:

df.write \
    .format("parquet") \
    .mode("overwrite") \
    .partitionBy( "symbol" ) \
    .save("/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Bronze")


### Bronze -> Silver

In [ ]:
from delta.tables import DeltaTable
from pyspark.sql.types import StructType, StructField, DateType, StringType, DecimalType, IntegerType

from pyspark.sql.functions import current_timestamp, sha2, concat_ws

# Paths for Bronze and Silver data
bronze_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Bronze"  # Bronze data stored in Parquet or Delta
silver_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver/delta-table"  # Target location for Silver Delta table
from pyspark.sql.functions import max, col,cast,unhex, hex



### Tranformt Bronze -> Silver
# 2. Load Bronze Data

df_bronze = spark.read.load(bronze_path).drop("symbol")
df_bronze = df_bronze.withColumn( "values", explode("values") )

df_bronze = df_bronze.select(
    col("meta").getItem("symbol").alias("Symbol"),
    col("meta").getItem("exchange").alias("ExchangeName"),
    col("meta").getItem("currency").alias("Currency"),
    col("meta").getItem("type").alias("Type"),
    col("meta").getItem("exchange_timezone").alias("ExchangeTimeZone"),
    col("values").getItem("volume").alias("Volume"),
    col("values").getItem("high").alias("High"),
    col("values").getItem("low").alias("Low"),
    col("values").getItem("close").alias("Close"),
    col("values").getItem("open").alias("Open"),
    col("values").getItem("datetime").alias("Date"))

df_bronze = df_bronze.dropDuplicates()

# df_bronze = df_bronze.withColumn("high", col("high")+20)




    


+------+------------+--------+----+----------------+--------+----------+---------+----------+---------+----------+-------------+-------------+--------------------+----------------------+--------------------+--------------------+--------------------+
|Symbol|ExchangeName|Currency|Type|ExchangeTimeZone|  Volume|      high|      Low|     Close|     Open|      Date|__CurrentFlag|__DeletedFlag|__ActivationDateTime|__DeactivationDateTime|      __lastmodified|           __HashKey|         __HashValue|
+------+------------+--------+----+----------------+--------+----------+---------+----------+---------+----------+-------------+-------------+--------------------+----------------------+--------------------+--------------------+--------------------+
|   VOO|        NYSE|     USD| ETF|America/New_York| 4098000| 430.32001|405.47000| 408.78000|405.70999|2021-09-23|         true|        false|2025-04-14 20:24:...|                  null|2025-04-14 20:24:...|00243ef129261e3e5...|1b4e302ab509b4956...|


In [117]:
spark.sql( """
          
          describe history delta.`/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver/delta-table`
          
          """).show(truncate=False)


+-------+-----------------------+------+--------+------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+-----------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|

25/04/14 20:24:17 WARN MergeIntoCommand: Merge source has SQLMetric(id: 22820, name: Some(number of source rows), value: 502) rows in initial scan but SQLMetric(id: 22821, name: Some(number of source rows (during repeated scan)), value: 0) rows in second scan


Merge operation complete.


In [114]:
df_silver = spark.read.format("delta").load(silver_path)

df_silver.createOrReplaceTempView("silver") 

spark.sql("select __hashkey, * from silver order by __hashkey , __ActivationDateTime desc ").show(truncate=False)

+----------------------------------------------------------------+------+------------+--------+----+----------------+--------+---------+---------+---------+---------+----------+-------------+-------------+--------------------------+----------------------+--------------------------+----------------------------------------------------------------+----------------------------------------------------------------+
|__hashkey                                                       |Symbol|ExchangeName|Currency|Type|ExchangeTimeZone|Volume  |High     |Low      |Close    |Open     |Date      |__CurrentFlag|__DeletedFlag|__ActivationDateTime      |__DeactivationDateTime|__lastmodified            |__HashKey                                                       |__HashValue                                                     |
+----------------------------------------------------------------+------+------------+--------+----+----------------+--------+---------+---------+---------+---------+--------

In [ ]:
df = spark.read.format("delta").load("/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver/delta-table")
df.schema

In [ ]:

# from pyspark.sql.types import StructType, StructField, DateType, StringType, DecimalType, IntegerType


# schema = StructType([StructField('Symbol', StringType(), True), 
#             StructField('ExchangeName', StringType(), True), 
#             StructField('Currency', StringType(), True), 
#             StructField('Type', StringType(), True), 
#             StructField('ExchangeTimeZone', StringType(), True), 
#             StructField('Volume', StringType(), True), 
#             StructField('High', StringType(), True), 
#             StructField('Low', StringType(), True), 
#             StructField('Close', StringType(), True), 
#             StructField('Open', StringType(), True), 
#             StructField('Date', StringType(), True)])

df_silver = spark.createDataFrame( df_bronze , schema = schema )

df_silver.show()

In [ ]:
df = spark.read.load("/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Bronze"); 

df_dim = df.select(
    col("meta").getItem("symbol").alias("Symbol"),
    col("meta").getItem("exchange").alias("ExchangeName"),
    col("meta").getItem("currency").alias("Currency"),
    col("meta").getItem("type").alias("Type"),
    col("meta").getItem("exchange_timezone").alias("ExchangeTimezone")
    ) 


# df_dim = df_dim.withColumn("__HashKey", sha2( col("Symbol"), 256)) 
# df_dim = df_dim.withColumn("__HashKey", sha2( concat_ws("|",  *businessColumnsList), 256)) 

 
# profile data
df_dim.limit(1).show(truncate = False) 


TypeIColumnsList

 

In [ ]:

df_flat_exploded = df_flat2.select(
                        col("symbol"),
                        explode( "values").alias("record"))
                        
                        # .withColumn( "volume" , col("record").getItem("volume") ) \
                        # .withColumn( "high" , col("record").getItem("high") ) \
                        # .withColumn( "low" , col("record").getItem("low") ) \
                        # .withColumn( "close" , col("record").getItem("close") ) \
                        # .withColumn( "open" , col("record").getItem("open") ) \
                        # .withColumn( "date" , col("record").getItem("datetime") ) \
                        #         .drop("record")


df_flat_exploded =   df_flat_exploded.select(
        "symbol",
        col("record").getItem("volume").alias("volume"),
        col("record").getItem("high").alias("high"),
        col("record").getItem("low").alias("low"),
        col("record").getItem("close").alias("close"),
        col("record").getItem("open").alias("open"),
        col("record").getItem("date").alias("date")
        
  )   
df_flat_exploded = df_flat_exploded.filter(  col("symbol")  == "VOO" ).distinct().count()     


print(df_flat_exploded)
# df_flat_exploded.show(truncate = False )